In [3]:
import os
import shutil
import numpy
import pandas
import c3d
from itertools import permutations

In [4]:
def get_angle(a,b,c):
    #ritorna angolo centrato in a
    #calcolo vettori
    v1 = a-b
    v2 = a-c
    m = v1.dot((v2).T/(numpy.linalg.norm(v1,axis=1)*numpy.linalg.norm(v2,axis=1)))
    d = numpy.diag(m)
    return (numpy.arccos(numpy.clip(d,-1.,1.)))

def get_angle_2d(a,b):
    #ritorno angoli con 3 coordinate
    x = (a-b)[:,0]
    y = (a-b)[:,1]
    z = (a-b)[:,2]
    #valore ritorno
    #stesso numero di punti con 6 componenti
    r = numpy.zeros((len(a),3))
    r[:,0] = numpy.arctan2(x,y)
    r[:,1] = numpy.arctan2(x,z)
    r[:,2] = numpy.arctan2(y,z)
    return r

In [17]:
#salviamo i file con indici crescenti da 0
#questo ha i seguenti vantaggi:
#possiamo generare con un xrange tutti i nomi di file
#possiamo prendere una permutazione semplicemente permutando il range
#usiamo l ordine naturale per la corrispondenza con la classe(file 3.c3d->indice 4 delle labels)

#PERCORSI RELATIVI!

#cartella base
#qui mi aspetto dentro le dat_folders
BASE_FOLDER = "data/"
#cartella con gli output
BASE_FOLDER_OUT = "processed/"
#classi 0-3(faremo +1 al massimo)
#cartelle 0-3
#dentro le data folders ho i rispettivi contenuti degli zip(cartelle con nome cognome)
DATA_FOLDERS = ["0/","1/","2/","3/"]
DATA_FOLDERS = ["0/"]

#classi associate alle data_folders
CLASSES = [i for i in range(len(DATA_FOLDERS))]
#parametri di interesse(presi da ferrari)
PARAMS = ["C7","REP","RUL","RASIS","RPSIS","RCA","RGT","RLE",
         "RFM","RA","LEP","LUL","LASIS","LPSIS","LCA","LGT","LLE",
         "LFM","LA"]
#angoli interessanti(a,b,c)
ANGLES_3D = [("LGT","LPSIS","LLE"),("LLE","LGT","LCA"),("LCA","LLE","LFM"),("LEP","LA","LUL"),("LEP","C7","LUL"),
          ("LLE","LASIS","LFM"),("LA","C7","LEP"),
          ("RGT","RPSIS","RLE"),("RLE","RGT","RCA"),("RCA","RLE","RFM"),("REP","RA","RUL"),("REP","C7","RUL"),
          ("RLE","RASIS","RFM"),("RA","C7","REP"),
         ("LPSIS","LGT","RGT"),("LASIS","LGT","RGT"),("LPSIS","LLE","RLE"),("C7","LA","RA"),("C7","LEP","REP"),
         ("RPSIS","LGT","RGT"),("RASIS","LGT","RGT"),("RPSIS","LLE","RLE"),("C7","LUL","RUL"),
         ("LASIS","C7","LPSIS"),("RASIS","C7","RPSIS"),("LA","LASIS","RASIS"),("RA","LASIS","RASIS")]

ANGLES_2D = [("LGT","LLE"),("LLE","LCA"),("LCA","LFM"),("LPSIS","LGT"),("LASIS","LGT"),
             ("LEP","LUL"),("LA","LEP"),("LA","LUL"),("C7","LA"),("C7","LPSIS"),
             ("RGT","RLE"),("RLE","RCA"),("RCA","RFM"),("RPSIS","RGT"),("RASIS","RGT"),
             ("REP","RUL"),("RA","REP"),("RA","RUL"),("C7","RA"),("C7","RPSIS"),
             ("LGT","RGT"),("LLE","RLE"),("LCA","RCA"),("LFM","RFM"),("LA","RA"),]

ANGLES_2D = [c for c in permutations(PARAMS,2)]

NUM_PARAMS = len(PARAMS)
#PARAMETRI CONTROLLO
TAKE_SEQ = True
REL = False
#angles3D->1 valore di angolo 3D
#angles2D->3 valori di angoli 2D sui piani XYZ
#points->3 valori XYZ
#speed->3 valori XYZ
#speed_module->1 valore di modulo di speed
#speed_angles_3D->1 valore di velocita angolare
#speed_angles_2D->3 valori di velocita angolare
#configuragioni relative(1 punto in meno)
CONF_REL = ["speed","speed_module","speed_angles_3D","speed_angles_2D"]

CONF = ['angles3D']

SWAP_AXIS = True
ALLOW_DIS = True
#frame vicon
FRAME_VICOM = 100
#frame che vogliamo ogni secondo
FRAME_PER_SECOND = 50
#guardiamo che sia divisibile
assert FRAME_VICOM % FRAME_PER_SECOND == 0
#campiono ogni FRAME_VICOM/FRAME_PER_SECOND
FRAME_COUNT = FRAME_VICOM/FRAME_PER_SECOND
#NUMERO SEQUENZE MASSIME
SEQ_NUM = 10
#LUNGHEZZA SEQUENZE
SEQ_LEN = 50
#DISPLACE TRA SEQUENZE CONSECUTIVE
DISPLACE = 50
#se usare MinMax
#minMax = True
#labels classi
labels_classes = []
#labels pazienti
labels_patients = []
invalid = 0
#controllo asse principale
top_axes_dist = {0:0,1:0,2:0}
#lista classi che salveremo in un file a parte
#lista perche non ho voglia di contare tutti i file,e piu lenta ma tanto lo lanciamo solo una volta
j = 0
len_step = []
for i in range(len(CLASSES)):
    #entro nella cartella classe
    folder_class = BASE_FOLDER+DATA_FOLDERS[i]
    print "start working in folder {}".format(folder_class)
    #per ogni cartella paziente estraggo i c3d
    for folder_patient in sorted(os.listdir(folder_class)):
        label_patient = int(folder_patient)
        folder_patient = folder_class+folder_patient
        #lista file
        for file_name in [file_name for file_name  in sorted(os.listdir(folder_patient)) if file_name[-3:] == "c3d"]:
            #questo file ha un errore nell header!
            if file_name == "114_8.c3d":
                continue
            #reader per leggere il c3d
            file_name = folder_patient+"/"+file_name
            reader = c3d.Reader(open(file_name,'rb'))
            first_frame = reader.header.first_frame
            last_frame = reader.header.last_frame
            
            context_header = [s.strip() for s in reader.groups['EVENT'].params['CONTEXTS'].string_array]
            label_header =[s.strip() for s in reader.groups['EVENT'].params['LABELS'].string_array]
            times_header = reader.groups['EVENT'].params['TIMES'].float_array
            times_header = times_header.flatten()
            times_header = times_header[times_header!=0]
            df = pandas.DataFrame(data={"context": context_header,"label":label_header, "times":times_header})
            df = df.sort_values(by="times")
            df.times = (df.times*100).astype("int32")
            #scartiamo se non ci sono abbastanza eventi interessanti
            if len(df.times) < 9 :
                continue
            #prendiamo dal occ
            occ = 0
            frame = min(df[(df.context== "Right") & (df.label== "Foot Strike")].iloc[occ].times,
                              df[(df.context== "Left") & (df.label== "Foot Strike")].iloc[occ].times)
            #scartiamo se il frame è negativo
            # TODO controlla gabri
            if frame-first_frame <0:
                continue

            #estraggo labels dei points
            params = [param.strip() for param in reader.get("POINT").get("LABELS").string_array]
            #guardo che ci siano tutti quelli del file matlab di Ferrari
            #sono 52 quelli che non l hanno
            #visto che ogni paziente ha piu prove non dovrebbe essere un problema
            if len([param for param in PARAMS if param not in params]) > 0:
                continue
            #ci sono tutti ma non so in che ordine!
            #mi servono gli indici dei param in params
            params = [params.index(param) for param in PARAMS]
            
            #li usero per accedere ai frame
            #ogni frame ha NUM_PARAMS*5 coordinate(xyz + precisione e altro)
            #devo salvare le sequenze
            #salvo un elemento in piu che usero come label
            #ho numero_sequenze*lunghezza_sequenza+1*numero_marker*3

            index_camp = FRAME_COUNT
            index =0
            #punti del file
            points = numpy.zeros((last_frame-first_frame+1,len(PARAMS),3))
            
            
            for k,point,analog in reader.read_frames():
                if k >= frame:
                    #se allow permetto di iniziare a campionare anche dopo frame
                    #ma non permetto comunque di saltare fotogrammi
                    #infatti funziona solo finche index e 0
                    if ALLOW_DIS and -1 in point[params,3:] and index == 0:
                        continue
                    #prendo un campione solo se index_camp = FRAME_COUNT
                    if index_camp == FRAME_COUNT:
                        #campiono

                        #se invalido esco
                        if -1 in point[params,3:]:
                            break
                            
                        points[index] = point[params,0:3]

                        
                        #incremento indice
                        index +=1
                        #camp da 1!!!
                        index_camp = 1
                    else:
                        index_camp +=1
            
            #prendo quelli validi
            points = points[0:index]
            #controllo che ci sia qualcosa
            if index == 0:
                print "non ci sono abbastanza dati per una sequenza in {}!".format(file_name)
                invalid +=1
                continue
            
            #controllo ordine assi
            if SWAP_AXIS:
                #questo lavora sempre sui relativi
                #ma va confrontato con i risultati di Gabri
                #sommo i marker ma in valore assoluto
                top = numpy.argmax(numpy.sum(numpy.abs(numpy.diff(points,axis=0)).reshape(-1,3),axis=0))
                #aggiorno contatori
                top_axes_dist[top]+=1
                #uno è il piu frequente, ma molte volte è zero
                #se è zero lo metto in posizione uno
                #due non c'è mai
                if top == 0:
                    points=points[:,:,[1,0,2]]
            
            #adesso cosa prendo dipende dalla CONF
            #la len e il numero di punti
            #ma se qualcuno ha valori relativizzati ho 1 in meno da considerare
            #per quelli che non sono relativi
            # viene considerato dopo
            arr = []
            for conf in CONF:
                if conf == "angles3D":
                    angles = numpy.zeros((len(points),len(ANGLES_3D),1))
                    for k,angle in enumerate(ANGLES_3D):
                        index_angle_a = PARAMS.index(angle[0])
                        index_angle_b = PARAMS.index(angle[1])
                        index_angle_c = PARAMS.index(angle[2])
                        angles[:,k,0] = get_angle(points[:,index_angle_a],points[:,index_angle_b],
                                                  points[:,index_angle_c])
                    temp= angles
                
                elif conf == "angles2D":
                    angles = numpy.zeros((len(points),len(ANGLES_2D),3))
                    for k,angle in enumerate(ANGLES_2D):
                        index_angle_a = PARAMS.index(angle[0])
                        index_angle_b = PARAMS.index(angle[1])
                        angles[:,k,:] = get_angle_2d(points[:,index_angle_a],points[:,index_angle_b])
                    temp = angles
                    
                elif conf == "points":
                    temp = points
                
                elif conf == "speed":
                    temp = numpy.diff(points,axis=0)
                
                elif conf == "speed_module":
                    #calcolo velocita
                    temp = numpy.diff(points,axis=0)
                    #modulo
                    temp = numpy.linalg.norm(temp,axis=2)[:,:,numpy.newaxis]
                
                elif conf == "speed_angles_3D":
                    angles = numpy.zeros((len(points),len(ANGLES_3D),1))
                    for k,angle in enumerate(ANGLES_3D):
                        index_angle_a = PARAMS.index(angle[0])
                        index_angle_b = PARAMS.index(angle[1])
                        index_angle_c = PARAMS.index(angle[2])
                        angles[:,k,0] = get_angle(points[:,index_angle_a],points[:,index_angle_b],
                                                  points[:,index_angle_c])
                    temp= numpy.diff(angles,axis=0)
                    
                elif conf == "speed_angles_2D":
                    angles = numpy.zeros((len(points),len(ANGLES_2D),3))
                    for k,angle in enumerate(ANGLES_2D):
                        index_angle_a = PARAMS.index(angle[0])
                        index_angle_b = PARAMS.index(angle[1])
                        angles[:,k,:] = get_angle_2d(points[:,index_angle_a],points[:,index_angle_b])
                    temp = numpy.diff(angles,axis=0)
                else:
                    raise NotImplementedError
                    
                arr.append(temp)
            #posso concatenare tutto
            #ma devo considerare le lunghezze
            min_len = min([len(el) for el in arr])
            
            points = numpy.concatenate([el[-min_len:] for el in arr],axis=1)
            print points.shape
            """""
            #potrei volere gli angoli
            if TAKE_ANGLE:
                if TAKE_ANGLE_3D:
                    angles = numpy.zeros((len(points),len(ANGLES_3D),1))
                    for k,angle in enumerate(ANGLES_3D):
                        index_angle_a = PARAMS.index(angle[0])
                        index_angle_b = PARAMS.index(angle[1])
                        index_angle_c = PARAMS.index(angle[2])
                        angles[:,k,:] = get_angle(points[:,index_angle_a],points[:,index_angle_b],points[:,index_angle_c])
                else:
                    angles = numpy.zeros((len(points),len(ANGLES_2D),6))
                    for k,angle in enumerate(ANGLES_2D):
                        index_angle_a = PARAMS.index(angle[0])
                        index_angle_b = PARAMS.index(angle[1])
                        angles[:,k,:] = get_angle_2d(points[:,index_angle_a],points[:,index_angle_b])
                           
                points = angles
            #potrei voler relativizzare
            if REL:
                points = numpy.diff(points,axis=0)
            """""
            #sequenze
            if TAKE_SEQ:    
                #sequenze
                #in ognuna tengo anche la label
                seq_numpy = numpy.zeros((SEQ_NUM,SEQ_LEN+1,points.shape[1],points.shape[2]))
                #contatori
                seq_index = 0
                base_index = 0
                while True:
                    #provo a prendere una sequenza
                    seq = points[base_index:base_index+SEQ_LEN]
                    if len(seq) != SEQ_LEN:
                        break
                    #provo a prendere la label
                    lab = points[base_index+SEQ_LEN:base_index+SEQ_LEN+1]
                    if len(lab) != 1:
                        break
                    #sono riuscito ad avere una nuova sequenza
                    seq_numpy[seq_index,:-1] = seq
                    seq_numpy[seq_index,-1] = lab
                    #avanzo di DISPLACE
                    base_index += DISPLACE
                    #avanzo seq_index
                    seq_index +=1
                    if seq_index == SEQ_NUM:
                        break
                #prendo solo quelle ottenute
                seq_numpy = seq_numpy[0:seq_index]



                #potrei essere qui ma non avere tutti i dati
                if seq_index == 0:
                    #non ho neanche una sequenza!
                    print "non ci sono abbastanza dati per una sequenza in {}!".format(file_name)
                    invalid +=1
                    continue
                else:
                    print "trovate {} sequenze in {}! ".format(seq_index,file_name)

                    data = seq_numpy[:,:-1,:,:]
                    labels = seq_numpy[:,-1:,:,:]
                    numpy.savez(BASE_FOLDER_OUT+"{}.c3d".format(j),data,labels)
                    #labels class
                    labels_classes.extend([i]*seq_index)
                    #labels patient
                    labels_patients.extend([label_patient]*seq_index)
                    j+=1
            else:
                    if len(points) == 0:
                        print "non ci sono abbastanza dati in {}!".format(file_name)
                        invalid +=1
                        continue
                    numpy.save(BASE_FOLDER_OUT+"{}.c3d".format(j),points)
                    #labels class
                    labels_classes.append(i)
                    #labels patient
                    labels_patients.append(label_patient)
                    j+=1
                 
    print "end working in folder {}".format(folder_class)
    print "-------"

numpy.save(BASE_FOLDER_OUT+"labels_classes",numpy.array(labels_classes))
numpy.save(BASE_FOLDER_OUT+"labels_patients",numpy.array(labels_patients))

print "invalid {}".format(invalid)
print "axis dist {}".format(top_axes_dist)
print "end"

start working in folder data/0/
(617, 722, 3)
trovate 10 sequenze in data/0/1/1_4.c3d! 
(550, 722, 3)
trovate 10 sequenze in data/0/1/1_7.c3d! 


KeyboardInterrupt: 